In [5]:
import sys
sys.path.append('../')

import random
import requests
import urllib.parse
from bs4 import BeautifulSoup
from objects.profile import Profile
import json
from collections import OrderedDict

from selenium import webdriver
from selenium_stealth import stealth
from selenium.webdriver.common.by import By
import time
from tqdm import tqdm
import pandas as pd

from data_sources.google_scholar import scrape_publication_details, scrape_google_scholar_profile, save_google_scholar_profile, save_publications_details

In [2]:
with open('./data/individual_profiles.csv', 'r') as f:
    ind_profile = json.load(f)
    
google_scholar_profile_url = []

for profile in ind_profile:
    if profile['google_scholar'] is not None:
        google_scholar_profile_url.append([profile['full_name'], profile['google_scholar']])
len(google_scholar_profile_url)

32

# Scraping Google Scholar Profile Page

In [ ]:
for profile in google_scholar_profile_url:
    name, url = profile[0], profile[1]

    g_profile = scrape_google_scholar_profile(url)
    save_google_scholar_profile(g_profile, output_filename=f'google_scholar_{name.lower().replace(" ","_")}.json')

    if random.uniform(0,1)>0.8:
        print('Sleeping')
        time.sleep(60)

# Scraping Citation Information

In [3]:
def create_driver(debug=False):

    options = webdriver.ChromeOptions()
    if debug==False:
        options.add_argument("--headless")
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    driver = webdriver.Chrome(
        options=options
    )
    stealth(driver,
            # user_agent=agent,
            languages=["en-US", "en"],
            vendor="Google Inc.",
            platform="Win32",
            webgl_vendor="Intel Inc.",
            renderer="Intel Iris OpenGL Engine",
            fix_hairline=True,
            )

    time.sleep(random.uniform(3,5))
    return driver


In [6]:
driver = create_driver(debug=True)

for j, profile in tqdm(enumerate(google_scholar_profile_url[1:])):
    name, url = profile[0], profile[1]
    with open(f'./data/google_scholar_{name.lower().replace(" ","_")}.json', 'r') as f:
        data = json.load(f)
    print(j)
    details = {}
    for i, publication in tqdm(enumerate(data['publications'])):
        key = publication['title_link']
        driver.get(key)
        time.sleep(random.uniform(3,6))
        details[key] = scrape_publication_details(key, driver)
        if random.uniform(0,1)>0.75:
            time.sleep(random.uniform(5,10))
        if random.uniform(0,1)>0.9:
            driver.quit()
            driver = create_driver(debug=False)
        
        if i%10==0:
            save_publications_details(details, f'google_scholar_{name.lower().replace(" ","_")}_publications.json')
    save_publications_details(details, f'google_scholar_{name.lower().replace(" ","_")}_publications.json')

0it [00:00, ?it/s]

0


5it [00:32,  6.43s/it]
0it [00:32, ?it/s]


KeyboardInterrupt: 

Sleeping
Sleeping
Sleeping
Sleeping
